In [1]:
import lightgbm as lgb
import neptune.new as neptune
from neptune.new.integrations.lightgbm import NeptuneCallback
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
import pandas_gbq as gbq
# from google.cloud import bigquery
# from google.cloud import bigquery_storage
from google.cloud import bigquery
from google.cloud import bigquery_storage_v1beta1

In [2]:
# Create run
run = neptune.init(
    project="eritsi/staple",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI0MTc5MWU2Ny1mZWJkLTRlYWYtYmY3Mi02MWY4YzdhMzg5ZDMifQ==",
    name="staple_train_trial",
    tags=["lgbm-integration", "train", "cls"]
)

Info (NVML): NVML Shared Library Not Found. GPU usage metrics may not be reported. For more information, see https://docs-legacy.neptune.ai/logging-and-managing-experiment-results/logging-experiment-data.html#hardware-consumption 


https://app.neptune.ai/eritsi/staple/e/STAP-10
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


In [3]:
# Create neptune callback
neptune_callback = NeptuneCallback(run=run)

In [4]:
# Prepare GCP setting & tables
project = "eri-sandbox"
bqclient = bigquery.Client(project=project, location="asia-northeast1")
bqstorageclient = bigquery_storage_v1beta1.BigQueryStorageClient()

/workspace/.pip-modules/lib/python3.8/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/workspace/.pip-modules/lib/python3.8/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/doc

In [5]:
sql = '''
            SELECT
              *
            FROM
              `eri-sandbox.staple_result_dghbc_submission_sales_Mar.datamart_dghbc_avoid_leak`
            ORDER BY
              product_code
          '''

In [6]:
# read bq table through bqstorage_client
df = (
    bqclient.query(sql)
    .result()
    .to_dataframe(
        bqstorage_client=bqstorageclient
    )
)

/workspace/.pip-modules/lib/python3.8/site-packages/pyarrow/util.py:43: FutureWarning: pyarrow.read_schema is deprecated as of 0.17.0, please use pyarrow.ipc.read_schema instead.
  warnings.warn(msg, FutureWarning)
/workspace/.pip-modules/lib/python3.8/site-packages/pyarrow/util.py:43: FutureWarning: pyarrow.read_record_batch is deprecated as of 0.17.0, please use pyarrow.ipc.read_record_batch instead.
  warnings.warn(msg, FutureWarning)


In [7]:
#
df.head(5)

,product_code,client_year,client_week_num,sales_count,comp_cluster,categories_small,categories_middle,categories_large,categories_xlarge,sb_class_small,...,pred_2mAVG_8w_5wLAG,pred_2mAVG_8w_6wLAG,pred_2mAVG_8w_10wLAG,pred_2mAVG_8w_12wLAG,pred_2mAVG_8w_14wLAG,pred_2mAVG,cumsum_sales_count_1w_2w,cumsum_sales_count_1w_3w,cumsum_sales_count_1w_4w,cumsum_sales_count_1w_5w
0,49502541,2017,32,826,12,3813100601,38131006,381310,381,11381310,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
1,49502541,2018,2,1038,12,3813100601,38131006,381310,381,11381310,...,0.0,0.0,0.0,0.0,0.0,0.0,2795.0,4133.0,5597.0,7124.0
2,49502541,2018,16,783,12,3813100601,38131006,381310,381,11381310,...,0.0,0.0,0.0,0.0,0.0,0.0,1564.0,2264.0,3179.0,4000.0
3,49502541,2018,37,1133,12,3813100601,38131006,381310,381,11381310,...,31324405.5,34251718.5,33734182.5,31998280.5,31092592.5,69899706.0,2501.0,3655.0,4416.0,5192.0
4,49502541,2019,7,1085,12,3813100601,38131006,381310,381,11381310,...,24065797.0,23142939.0,21235482.5,20237886.0,18060721.0,56525052.5,2275.0,3349.0,4327.0,5226.0


In [8]:
# Prepare data
X, y = load_digits(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=123
)
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

In [9]:
# Define parameters
params = {
    "boosting_type": "gbdt",
    "objective": "multiclass",
    "num_class": 10,
    "metric": ["multi_logloss", "multi_error"],
    "num_leaves": 21,
    "learning_rate": 0.05,
    "feature_fraction": 0.9,
    "bagging_fraction": 0.8,
    "bagging_freq": 5,
    "max_depth": 12,
}

In [10]:
# Train the model
gbm = lgb.train(
    params,
    lgb_train,
    num_boost_round=200,
    valid_sets=[lgb_train, lgb_eval],
    valid_names=["training", "validation"],
    callbacks=[neptune_callback],
)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001032 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 834
[LightGBM] [Info] Number of data points in the train set: 1437, number of used features: 53
[LightGBM] [Info] Start training from score -2.335839
[LightGBM] [Info] Start training from score -2.273101
[LightGBM] [Info] Start training from score -2.321553
[LightGBM] [Info] Start training from score -2.259678
[LightGBM] [Info] Start training from score -2.335839
[LightGBM] [Info] Start training from score -2.293579
[LightGBM] [Info] Start training from score -2.343059
[LightGBM] [Info] Start training from score -2.273101
[LightGBM] [Info] Start training from score -2.350332
[LightGBM] [Info] Start training from score -2.246432
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f